In [6]:
import torch
import torch.utils.data
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [7]:
tweet_data = pd.read_csv("text_quality.csv", encoding = "ISO-8859-1")
tweet_data = tweet_data.drop_duplicates(subset="text")

tweet_data.head(5)

,text,no_chars,no_words,no_unique_words,word_difference,number_hashtags,number_mentions,number_urls,client_sources,listed_count,verified,geo_enabled,statuses_count,followers_count,favourites_count,user_lang,friends_count,class
0,#DanAndPhilBRITs Make sure they zoom in on Lou...,4.499810,2.639057,2.639057,0.000000,1.098612,0.000000,0.000000,Twitter for iPhone,4,False,True,5604,259,4880,nl,446,reg
1,I voted for DRAG ME DOWN by @onedirection for ...,4.828314,2.890372,2.833213,0.693147,1.098612,0.693147,0.693147,Mobile Web,0,False,False,198,64,370,en,380,cont
2,I voted for DRAG ME DOWN by @onedirection for ...,4.867534,2.995732,2.944439,0.693147,1.098612,0.693147,0.693147,Twitter Web Client,4,False,False,3843,794,2401,en,334,cont
3,@brits @danisnotonfire @AmazingPhil #BRITs #Da...,4.304065,2.079442,2.079442,0.000000,1.098612,1.386294,0.000000,Twitter for iPhone,9,False,True,24432,3608,29325,en,2974,cont
4,Hello from RUSSIA <ed><U+00A0><U+00BC><ed><U+0...,5.863631,2.708050,2.708050,0.000000,1.098612,1.098612,0.000000,Twitter for iPhone,3,False,False,3274,45,31,ru,21,cont


In [8]:
# Proportion of classes

reg_class = tweet_data[(tweet_data["class"] == "reg")]
cont_class = tweet_data[(tweet_data["class"] == 'cont')]

# tweet_data['class'] = tweet_data['class'].map({'reg': [1, 0], 'cont': [0, 1]})
# tweet_data['class'] = tweet_data['class'].map({'reg': 2, 'cont': 1})
tweet_data['class'] = tweet_data['class'].map({'reg': 1, 'cont': 0})

print("Regular Tweets %: ", round((len(reg_class) / (len(reg_class) + len(cont_class))) * 100, 2))
print("Contaminated Tweets %: ", round((len(cont_class) / (len(reg_class) + len(cont_class))) * 100, 2))

Regular Tweets %:  74.8
Contaminated Tweets %:  25.2


In [9]:
inputData = tweet_data.drop(["text","client_sources","user_lang","class"], axis=1).as_matrix() #np.float32()
outputData = tweet_data["class"].as_matrix()

# trainloader_input = torch.utils.data.DataLoader(inputData, batch_size=4, shuffle=True, num_workers=8)
# trainloader_output = torch.utils.data.DataLoader(outputData, batch_size=4, shuffle=True, num_workers=8)
# print(trainloader_output)

In [10]:
# Example NN using torch

from torch.autograd import Variable
import torch.nn as nn

# N is batch size; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.
N, D_in, H, D_out = 1, inputData.shape[1], 8, 2

# Set our variables up
x_data = inputData # Variable(torch.from_numpy(inputData))
y_data = outputData # Variable(torch.from_numpy(outputData), requires_grad=False)


class TwoLayerNet(torch.nn.Module):
    def __init__(self, D_in, H, D_out):
        """
        In the constructor we instantiate two nn.Linear modules and assign them as
        member variables.
        """
        super(TwoLayerNet, self).__init__()
        self.linear1 = torch.nn.Linear(D_in, H)
        self.linear2 = torch.nn.Linear(H, D_out)

    def forward(self, x):
        """
        In the forward function we accept a Variable of input data and we must return
        a Variable of output data. We can use Modules defined in the constructor as
        well as arbitrary operators on Variables.
        """
        h_relu = self.linear1(x).clamp(min=0)
        y_pred = self.linear2(h_relu)
        return y_pred


model = TwoLayerNet(D_in, H, D_out)

In [13]:
# Optimize and error calculation

import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

for t in range(10):
    # Forward pass: Compute predicted y by passing x to the model
    running_loss = 0.0
    for i, data in enumerate(zip(x_data, y_data), 0):
    #for i, data in enumerate(zip(trainloader_input, trainloader_output), 0):
        
        x, y = data
#         x = Variable(x)
#         y = Variable(torch.stack(y, 0))
#         x = np.long32(x).tolist()
        
#         x = Variable(torch.from_numpy(x))
        
        x = Variable(torch.FloatTensor([x]))
        y = Variable(torch.IntTensor([int(y)]))
        #y = Variable(torch.FloatTensor(y)) #[int(y)]))
        
        y_pred = model(x)

        # Compute and print loss
        loss = criterion(y_pred, y.long())
#         print(t, loss.data[0])

        # Zero gradients, perform a backward pass, and update the weights.
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
        # print statistics
        running_loss += loss.data[0]
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (t + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

[1,  2000] loss: 0.897
[1,  4000] loss: 0.528
[1,  6000] loss: 0.525
[1,  8000] loss: 0.518
[1, 10000] loss: 0.520
[1, 12000] loss: 0.695
[1, 14000] loss: 0.552
[1, 16000] loss: 0.511
[1, 18000] loss: 0.549
[1, 20000] loss: 0.513
[1, 22000] loss: 0.648
[1, 24000] loss: 0.472
[1, 26000] loss: 0.430
[1, 28000] loss: 0.504
[1, 30000] loss: 0.526
[1, 32000] loss: 0.429
[1, 34000] loss: 0.513
[2,  2000] loss: 0.641
[2,  4000] loss: 0.528
[2,  6000] loss: 0.525
[2,  8000] loss: 0.518
[2, 10000] loss: 0.520
[2, 12000] loss: 0.695
[2, 14000] loss: 0.552
[2, 16000] loss: 0.511
[2, 18000] loss: 0.549
[2, 20000] loss: 0.513
[2, 22000] loss: 0.648
[2, 24000] loss: 0.472
[2, 26000] loss: 0.430
[2, 28000] loss: 0.504
[2, 30000] loss: 0.526
[2, 32000] loss: 0.429
[2, 34000] loss: 0.513
[3,  2000] loss: 0.641
[3,  4000] loss: 0.528
[3,  6000] loss: 0.525
[3,  8000] loss: 0.518
[3, 10000] loss: 0.520
[3, 12000] loss: 0.695
[3, 14000] loss: 0.552
[3, 16000] loss: 0.511
[3, 18000] loss: 0.549
[3, 20000] 

In [3]:
# -*- coding: utf-8 -*-
import torch
from torch.autograd import Variable


class TwoLayerNet(torch.nn.Module):
    def __init__(self, D_in, H, D_out):
        """
        In the constructor we instantiate two nn.Linear modules and assign them as
        member variables.
        """
        super(TwoLayerNet, self).__init__()
        self.linear1 = torch.nn.Linear(D_in, H)
        self.linear2 = torch.nn.Linear(H, D_out)

    def forward(self, x):
        """
        In the forward function we accept a Variable of input data and we must return
        a Variable of output data. We can use Modules defined in the constructor as
        well as arbitrary operators on Variables.
        """
        h_relu = self.linear1(x).clamp(min=0)
        y_pred = self.linear2(h_relu)
        return y_pred


# N is batch size; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.
N, D_in, H, D_out = 1, 1000, 100, 2

# Create random Tensors to hold inputs and outputs, and wrap them in Variables
x = Variable(torch.randn(N, D_in))
y = Variable(torch.randn(N, D_out), requires_grad=False)

# Construct our model by instantiating the class defined above
model = TwoLayerNet(D_in, H, D_out)

# Construct our loss function and an Optimizer. The call to model.parameters()
# in the SGD constructor will contain the learnable parameters of the two
# nn.Linear modules which are members of the model.
criterion = torch.nn.MSELoss(size_average=False)
optimizer = torch.optim.SGD(model.parameters(), lr=1e-4)
for t in range(100):
    # Forward pass: Compute predicted y by passing x to the model
    y_pred = model(x)

    # Compute and print loss
    loss = criterion(y_pred, y)
    print(t, loss.data[0])

    # Zero gradients, perform a backward pass, and update the weights.
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

0 2.5235109329223633
1 2.334049701690674
2 2.165907144546509
3 2.0116193294525146
4 1.876344084739685
5 1.7554758787155151
6 1.6425457000732422
7 1.537375807762146
8 1.4427156448364258
9 1.3539345264434814
10 1.271077036857605
11 1.1933000087738037
12 1.1202627420425415
13 1.0524216890335083
14 0.9896336793899536
15 0.9316871166229248
16 0.8776638507843018
17 0.8269673585891724
18 0.7791957259178162
19 0.7341635227203369
20 0.6917116045951843
21 0.6516913175582886
22 0.6139622926712036
23 0.5783936381340027
24 0.5448616743087769
25 0.5132501721382141
26 0.48345011472702026
27 0.4553581178188324
28 0.4288780689239502
29 0.40391805768013
30 0.38039201498031616
31 0.3582189977169037
32 0.3373225927352905
33 0.3184129297733307
34 0.3018184006214142
35 0.28607842326164246
36 0.27114975452423096
37 0.25699126720428467
38 0.24356409907341003
39 0.23083099722862244
40 0.2187567949295044
41 0.20751559734344482
42 0.19706718623638153
43 0.18713364005088806
44 0.17769041657447815
45 0.16871404647

In [171]:
torch.FloatTensor([x])

RuntimeError: already counted a million dimensions in a given sequence. Most likely your items are also sequences and there's no way to infer how many dimension should the tensor have